## Version 11

Updates from v10:
- Added option to read in previously processed countries from a separate folder 
- Added in custom code for Sudan / South Sudan in adm0_processing_2016 function

<a id='Instruction'></a>
### Instructions
- [Change path names here](#Paths)
- [If there are previously processed results, read them in here](#Previous)
- [Run all cells until here, where you can then select the appropriate section](#TOC)

## Google CoLab: Install packages & mount drive

In [1]:
# !pip install country_converter

In [2]:
# !pip install geopandas

In [3]:
# !pip install rasterio

In [4]:
# !pip install pycountry

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

## Import packages

In [1]:
import pandas as pd 

In [2]:
import os, time
import country_converter as coco

import shapely
from osgeo import gdal
import pandas as pd 
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path 
import pycountry 

import rasterio as rs
from rasterio.plot import show
from rasterio.enums import Resampling
from rasterio.plot import plotting_extent
from rasterio.merge import merge
import rasterio.mask
from rasterio import shutil as rio_shutil
from rasterio.vrt import WarpedVRT

def tPrint(s):
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

<a id='Paths'></a>
## Defining paths
*Change the path in the first cell to the GLOBAL folder containing file contents.*  
*Change the path in the second cell to the a location where a few folders can be created to save outputs.*

After changing paths:
- [If there are previously processed results, read them in here](#Previous)
- [Otherwise, run all cells until here, where you can then select the appropriate section](#TOC)


In [3]:
#path = Path('/home/public/Data/GLOBAL') #Change Path 
path = Path('/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL') #Change Path 
#path = Path('/content/drive/My Drive/World_Bank_Project/GLOBAL') #Change Path 

In [4]:
path

PosixPath('/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL')

In [5]:
#outputPath = Path('/home/wb411133/data/Projects/FATHOM') #Change Path 
outputPath = Path('/Users/meldasalhab/Google_Drive/World_Bank_Project') #Change Path 
# outputPath = Path('/content/drive/My Drive/World_Bank_Project') #Change Path 


In [6]:
# World Admin
worldAdmin_Folder = 'GMGD world admin'
admin_File = 'newold_geo_code2_povdata_v6.shp'

reproj_Folder = 'GMGD_reproj'
admin_File_proj = 'newold_geo_code2_povdata_v6_4326.shp'

# Pluvial and fluvial flood files (Fathom) - 2019
flood_Folder = 'FLOOD_SSBN'
flood_subFolder_2019 = 'v2_2019'
fluvialFlood_Folder = 'fluvial_undefended'
fluvialFlood_FileName = 'FU_1in100.tif' 
pluvialFlood_Folder = 'pluvial'
pluvialFlood_FileName = 'P_1in100.tif' 

# Pluvial and fluvial flood files (Fathom) - 2016
flood_subFolder_2016 = 'v1_2016'
## The pluvial and fluvial folder names are prepended with each country's ISO2 code
## This is managed in the 2016 analysis loop

# Coastal flood files
coastalFlood_Folder = 'Coastal_flood'
coastalFlood_File = "ss_muis_rp0100m.tif"

# Population data
pop_Folder = 'Population'
pop_subFolder = 'WorldPop_PPP_2020'
pop_subsubFolder = 'MOSAIC_ppp_prj_2020'

# Folder paths
flood_path_2019 = path / flood_Folder / flood_subFolder_2019 
flood_path_2016 = path / flood_Folder / flood_subFolder_2016 
population_path = path / pop_Folder / pop_subFolder / pop_subsubFolder 

# Global files paths
adm_path_reproj = path / worldAdmin_Folder / reproj_Folder / admin_File_proj
adm_path = path / worldAdmin_Folder / admin_File
coastalFlood_path = path / coastalFlood_Folder / coastalFlood_File
 
print(flood_path_2019)
print(flood_path_2016)
print(population_path)
print(adm_path)
print(coastalFlood_path)

/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL/FLOOD_SSBN/v2_2019
/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL/FLOOD_SSBN/v1_2016
/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL/Population/WorldPop_PPP_2020/MOSAIC_ppp_prj_2020
/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL/GMGD world admin/newold_geo_code2_povdata_v6.shp
/Users/meldasalhab/Google_Drive/World_Bank_Project/GLOBAL/Coastal_flood/ss_muis_rp0100m.tif


## Creating folders for intermediate & final outputs


In [7]:
# Creating folder for all outputs
output_Folder = 'Outputs'
output_FolderPath = outputPath / output_Folder
try: 
    output_FolderPath.mkdir()
except FileExistsError:
    print('Already exists')
    

Already exists


In [8]:
# Creating folder for raster outputs
Raster_outputFolder = 'FloodPop_Countries'
Raster_outputFolderPath = output_FolderPath / Raster_outputFolder
try: 
    Raster_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')
    

Already exists


In [9]:
# Creating folder for coastal flood country files
cFlood_outputFolder = 'CoastalFlood_Countries'
cFlood_outputFolderPath = output_FolderPath / cFlood_outputFolder
try: 
    cFlood_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [10]:
# Creating folder for aggregated flood country files (fluvial + pluvial + coastal)
Flood_outputFolder = 'Flood_Countries'
Flood_outputFolderPath = output_FolderPath / Flood_outputFolder
try: 
    Flood_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [11]:
# Creating folder for fluvial and pluvial merged flood files
PopAligned_outputFolder = 'Pop_aligned'
PopAligned_outputFolderPath = output_FolderPath / PopAligned_outputFolder
try: 
    PopAligned_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [12]:
# Creating folder for final results
Results_outputFolder = 'Results'
Results_outputFolderPath = output_FolderPath / Results_outputFolder
try: 
    Results_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


In [13]:
# Creating folder for fluvial and pluvial merged flood files
FP_outputFolder = 'FluvialPluvial'
FP_outputFolderPath = output_FolderPath / FP_outputFolder
try: 
    FP_outputFolderPath.mkdir()
except FileExistsError:
    print('Already exists')

Already exists


<a id='Previous'></a>
## Previously Processed Countries
If there are any previously processed countries, uncomment the second cell below. 
Otherwise, make sure the cell is commented out.

After commenting/uncommenting the previously processed:
- Run all cells until [here](#TOC), where you can then select the appropriate section


In [18]:
previouslyProcessed = []

In [21]:
# Comment out below if previously processed results do not exist or if you wish to rewrite
July_processed = outputPath / 'Results_20200714' /  "processed_countries.shp"
processed = gpd.read_file(July_processed) 

# processed = gpd.read_file(Results_outputFolderPath / "processed_countries.shp")
previouslyProcessed = processed['ISO3'].unique()

## Flood Bins
0. No risk: x = 0
1. Limited risk: x <= 0.15
2. Moderate risk: 0.15 < x <= 0.5
3. High risk: 0.5 < x <= 1.5
4. Very high risk: x > 1.5
5. Water body: x >= 999


In [14]:
flood_bins = [1e-10,0.15,0.5,1.5,998,10000] 
# Right=True when digitizing to include right bin edge
# Note: 0 contains flood depths of 0 up to 0.0000000001 -> no flood
numberCategories = len(flood_bins)

## Results dataframe

In [15]:
# An empty dataframe to store the results of each loop
result = pd.DataFrame(columns = ['OBJECTID',
                                '0-NoRiskPop',
                                '1-LowRiskPop', 
                                '2-ModerateRiskPop', 
                                '3-HighRiskPop', 
                                '4-VeryHighRiskPop', 
                                '5-WaterBodyPop',
                                'ISO3']).set_index('OBJECTID')

## Admin shapefiles

In [1]:
# If previously reprojected file exists:
adm = gpd.read_file(adm_path_reproj) #already reprojected


In [18]:
# If not:
#adm = gpd.read_file(adm_path)
#adm = adm[['code','sample','region','geo_code2','geometry']]
# Reproject to EPSG:4326
#adm = adm.to_crs('EPSG:4326')

In [108]:
# Correcting country codes
adm['ISO3'] = adm['code']
adm.ISO3 = adm.ISO3.replace('ADO', 'AND')
adm.ISO3 = adm.ISO3.replace('IMY', 'IMN')
adm.ISO3 = adm.ISO3.replace('URU', 'URY')
adm.ISO3 = adm.ISO3.replace('VNZ', 'VEN')
adm.ISO3 = adm.ISO3.replace('MOR', 'MAR')
#Kosovo WorldPop was originally KOS, then manually changed to XKX on the server

## Matching adm country names to folder/file names

In [109]:
# Get country codes from adm df
codeDF = adm[['ISO3']]
codeDF = codeDF.drop_duplicates(subset ="ISO3")
codeDF = codeDF.sort_values(by=['ISO3'])

# Put adm country names in list for easier comparison
admCodeList = codeDF.ISO3.values.tolist()
admList = coco.convert(names=admCodeList, to='name_short')
#admList.sort()

# Add country names to df
codeDF['ADM0_NAME'] = admList
adm = adm.merge(codeDF, on='ISO3', how='left')

In [25]:
# Get folder names for the Fathom data
list2019 = [os.path.basename(str(x)) for x in flood_path_2019.iterdir() if x.is_dir()]
list2016 = [os.path.basename(str(x)) for x in flood_path_2016.iterdir() if x.is_dir()]

In [27]:
# Match country names from lists - 2019
matching_dict_2019 = coco.match(admList, list2019);

In [29]:
# Match country names from lists - 2016
matching_dict_2016 = coco.match(admList, list2016);

## Correcting specific cases

In [110]:
# Sierra Leone has two folders, selecting one
if type(matching_dict_2019['Sierra Leone']) is list:
    matching_dict_2019['Sierra Leone'] = 'Sierra Leone'

In [111]:
# There is only one flood folder for Sudan so setting South Sudan to also read from it
if matching_dict_2016['South Sudan'] == 'not_found':
    matching_dict_2016['South Sudan'] = 'Sudan'

In [112]:
# Fiji flood files are separated into Fiji_west and Fiji_east
for name in flood_path_2019.glob('Fiji_west'):
    fluvialFolder = flood_path_2019 / name / fluvialFlood_Folder
    FJfluvial = list(fluvialFolder.rglob('*100.tif'))
    
for name in flood_path_2019.glob('Fiji_east'):
    pluvialFolder = flood_path_2019 / name / pluvialFlood_Folder
    FJ_E_pluvial = list(pluvialFolder.rglob('*100.tif'))
    
for name in flood_path_2019.glob('Fiji_west'):
    pluvialFolder = flood_path_2019 / name / pluvialFlood_Folder
    FJ_W_pluvial = list(pluvialFolder.rglob('*100.tif'))

In [113]:
# Reorder so geometry is the last col
adm = adm[['ADM0_NAME','code','ISO3','sample','region','geo_code2','geometry']]

In [114]:
# from previous runs - combined 2016 and 2019 ,'PYF_1992'
bad_adm1 = ['FSM_2','TLS_201', 'BHR_2229','CAN_2254',
            'MAR_502','RUS_391','RUS_421','RUS_443','RUS_453',
            'RUS_2252','USA_759','HND_2117','MAR_502']

# Functions


In [55]:
def get_FP_2019(flood_name, path_name, pfFlood_Folder):
  floodList = []
  for name in path_name.glob(flood_name):
      floodFolder = path_name / name / pfFlood_Folder
      if len(list(floodFolder.rglob('*100.tif'))) == 0:
          # Large countries have multiple FU_1in100-x.tif files (x = tile number)
          floodList = list(floodFolder.rglob('*100_tile_?.tif'))
      else:
          # Small countries have one FU_1in100.tif file
          floodList = list(floodFolder.rglob('*100.tif'))
  return floodList


In [56]:
def get_FP_2016(flood_name, path_name, pfFlood_Folder):
  # Creating list of fluvial flood files
  floodList = []
  for name in path_name.glob(flood_name):
          floodFolder = path_name / name / pfFlood_Folder
          floodList = list(floodFolder.rglob('*.tif'))
  return floodList

In [57]:
def open_files_in_list(fileList):
    outputList = []
    for fp in fileList:
        src = rs.open(fp)
        outputList.append(src) 
    return outputList

In [58]:
def merge_export_adm0_FP_files(flood_files_list, outpath, countryCode, FP):
    if len(flood_files_list) >=1:
      # if multiple flood files exist, merge then export
      if len(flood_files_list) >1: 
          # Merge flood files.
          floodArray, out_trans = merge(flood_files_list, method='max')
          tPrint(f'{countryCode}: Country level {FP} merged') 
          ## Update the metadata
          out_meta = flood_files_list[0].meta.copy()
          out_meta.update({"height": floodArray.shape[1], 
                            "width": floodArray.shape[2],
                            "transform": out_trans})
      elif len(flood_files_list) ==1: 
          floodArray = flood_files_list[0].read()
          out_meta = flood_files_list[0].meta.copy()
          tPrint(f'{countryCode}: Only one {FP} tile. No merge needed')
      # export as a new geotiff 
      floodName = f"{FP}_{countryCode}.tif"
      merged_outputfile = outpath / floodName
      with rs.open(merged_outputfile, 'w', **out_meta, compress = 'LZW', tiled=True) as dest:
          dest.write(floodArray)
      tPrint(f'{countryCode}: Country level {FP} exported') 
    else:
      tPrint(f'{countryCode}: No {FP} files found.')
      merged_outputfile = outpath # fix or use this in next loop
    return merged_outputfile


In [59]:
def adm0_processing_2019(flood_name):
    # all other variables used are global
    ###################### Fluvial Adm0 ######################
    if flood_name == ['Fiji_east', 'Fiji_west']: 
      fluvialList = FJfluvial 
    else: 
      fluvialList = get_FP_2019(flood_name, flood_path_2019, fluvialFlood_Folder) 

    fluvialFiles = open_files_in_list(fluvialList)      
    Fluvial_outputfile = merge_export_adm0_FP_files(fluvialFiles, FP_outputFolderPath, countryCode, 'fluvial')

    ###################### Pluvial Adm0 ######################
    # Create list of pluvial flood files
    if flood_name == ['Fiji_east', 'Fiji_west']:
      pluvialList = FJ_E_pluvial + FJ_W_pluvial
    else:
      pluvialList = get_FP_2019(flood_name, flood_path_2019, pluvialFlood_Folder)

    pluvialFiles = open_files_in_list(pluvialList)  
    Pluvial_outputfile = merge_export_adm0_FP_files(pluvialFiles, FP_outputFolderPath, countryCode, 'pluvial')

    return Fluvial_outputfile, Pluvial_outputfile


In [84]:
def adm0_processing_2016(flood_name):
    # all other variables used are global
    ###################### Fluvial Adm0 ######################
    # South Sudan - Need to use Sudan fluvial and pluvial files 
    if countryCode == 'SSD': 
        fluvialFlood_Folder_2016 = 'SD_fluvial_undefended'
    else:
        fluvialFlood_Folder_2016 = f'{countryNameInfo.alpha_2}_fluvial_undefended'
        
    fluvialList = get_FP_2016(flood_name, flood_path_2016, fluvialFlood_Folder_2016)
    fluvialFiles = open_files_in_list(fluvialList) 
    Fluvial_outputfile = merge_export_adm0_FP_files(fluvialFiles, FP_outputFolderPath, countryCode, 'fluvial')

    ###################### Pluvial Adm0 ######################
    # South Sudan - Need to use Sudan fluvial and pluvial file
    if countryCode == 'SSD':
        pluvialFlood_Folder_2016 = 'SD_pluvial_undefended'
    else:
        pluvialFlood_Folder_2016 = f'{countryNameInfo.alpha_2}_pluvial_undefended'
        
    pluvialList = get_FP_2016(flood_name, flood_path_2016, pluvialFlood_Folder_2016)
    pluvialFiles = open_files_in_list(pluvialList)  
    Pluvial_outputfile = merge_export_adm0_FP_files(pluvialFiles, FP_outputFolderPath, countryCode, 'pluvial')
    
    return Fluvial_outputfile, Pluvial_outputfile

In [61]:
def get_adm1_index(df_adm1):
    adm1Index = str(df_adm1.index)
    adm1Index = adm1Index.partition("[")[2].partition("]")[0]
    return adm1Index

In [62]:
def crop_to_shp(path_name, df, filePrefix):
    raster = rs.open(path_name)
    # Mask to shapefile
    out_array, out_trans = rs.mask.mask(dataset=raster, shapes=df.geometry, crop=True)

    out_meta = raster.meta.copy()
    out_meta.update({"height": out_array.shape[1], 
                      "width": out_array.shape[2],
                      "transform": out_trans})
    tPrint(f"{countryCode}_{adm1Index} - {filePrefix} cropped to adm 1")
    return out_array, out_meta
    

In [63]:
def export_raster_to_geotiff(array, meta, path, fileName, filePrefix):
    Cropped_outputfile = path / fileName
    with rs.open(Cropped_outputfile, 'w', **meta, compress = 'LZW', tiled=True) as dest:
        dest.write(array)
    return Cropped_outputfile
    tPrint(f"{countryCode}_{adm1Index} - {filePrefix} exported")
    

In [64]:
def vrtWarp(inpath, outpath, vrt_settings, country_code, adm_code, filePrefix):
    with rs.open(inpath) as src:
      with WarpedVRT(src, **vrt_settings) as vrt:
          data = vrt.read()
          # Process the dataset in chunks. 
          for _, window in vrt.block_windows(): data = vrt.read(window=window)
          # Export the aligned data 
          fileName = f"{filePrefix}_{country_code}_{adm_code}.tif"
          Aligned_outputfile = outpath / fileName        
          rio_shutil.copy(vrt, Aligned_outputfile, driver='GTiff')
    tPrint(f"{countryCode}_{adm1Index} - {filePrefix} aligned")
    return Aligned_outputfile


In [65]:
def convert_to_bool_int_array(array):
      listNumberCat = range(numberCategories)
      list_IntBool_floodCat = []
      # Create a boolean int array for each category
      for i in listNumberCat:
          intArray = (array == i).astype(dtype = np.int8, copy = False)
          list_IntBool_floodCat.append(intArray)
      tPrint(f"{countryCode}_{adm1Index} - Flood converted to bool int arrays")
      return list_IntBool_floodCat


In [66]:
def multiply_array_list(arrayList, array):
      flood_pop = []
      for i in range(len(arrayList)):
          flood_pop.append(np.multiply(arrayList[i], array))
      flood_pop = (np.vstack((flood_pop)))
      tPrint(f"{countryCode}_{adm1Index} - Flood multiplied by pop")
      return flood_pop


In [67]:
def array3d_sum(array):
      Flood_pop_sums = {
        '0-NoRiskPop': np.sum(array[0]),
        '1-LowRiskPop': np.sum(array[1]),
        '2-ModerateRiskPop': np.sum(array[2]),
        '3-HighRiskPop': np.sum(array[3]),
        '4-VeryHighRiskPop': np.sum(array[4]),
        '5-WaterBodyPop': np.sum(array[5])
      }
      tPrint(f"{countryCode}_{adm1Index} - Checksum: {np.sum(array)}")
      return Flood_pop_sums

In [68]:
def sums_to_df(sums, df):
      for key,value in sums.items() :
        df.loc[int(adm1Index), key] = value

      # rounding the values to x decimal places
      df.iloc[:,-6:] = df.iloc[:,-6:].apply(lambda x: round(x, 2));

      # Keeping only results cols to make merge easy
      df = df.iloc[:,-6:] 
      df['ISO3'] = countryCode
      return df



In [69]:
def adm1_processing(adm1, resultDF):
    ###################### Fluvial Adm1 ######################
    # crop
    floodArray, out_meta = crop_to_shp(Fluvial_outputfile, adm1, 'Fluvial')
    # export
    fluvialCroppedName = f"FU_{countryCode}_{adm1Index}.tif"
    FluvialCropped_outputfile = export_raster_to_geotiff(floodArray, out_meta, FP_outputFolderPath, 
                                                        fluvialCroppedName, 'Fluvial')
    
    # ###################### Pluvial Adm1 ######################
    # crop
    floodArray, out_meta = crop_to_shp(Pluvial_outputfile, adm1, 'Pluvial')
    # export
    pluvialCroppedName = f"PU_{countryCode}_{adm1Index}.tif"
    PluvialCropped_outputfile = export_raster_to_geotiff(floodArray, out_meta, FP_outputFolderPath, 
                                                        pluvialCroppedName, 'Pluvial')
    
    ###################### WarpedVRT ######################
    # Vritually warp the pop and coastal flood files to the fluvial and pluvial
    vrt_options ={
        'resampling': Resampling.nearest,
        'crs': out_meta['crs'],
        'transform' : out_meta['transform'],
        'height':floodArray.shape[1],
        'width': floodArray.shape[2]
    }

    ###################### Population ######################
    # Get population file
    pop_File = " "
    countryAlpha3 = f'*{countryCode}*'
    for name in population_path.glob(countryAlpha3):
        pop_File = name
    if pop_File != " ":
        # Align using Warped VRT
        popAligned_outputfile = vrtWarp(pop_File, PopAligned_outputFolderPath, vrt_options, countryCode, adm1Index, 'popAligned')
        pop = rs.open(popAligned_outputfile)
        # crop
        popArray, out_meta = crop_to_shp(popAligned_outputfile, adm1, 'Pop')
        # change dtype to reduce memory
        popArray = popArray.astype('float32', copy=False)
        out_meta.update({'dtype': 'float32'})
        # export
        popCropped = f"PopCropped_{countryCode}_{adm1Index}.tif"
        export_raster_to_geotiff(popArray, out_meta, PopAligned_outputFolderPath, popCropped, 'popCropped')
        # Remove negatives
        popArray[popArray < 0] = 0
        # Print population sum for country to compare against final results
        tPrint(f"{countryCode}_{adm1Index} - Pop aligned & cropped. Total Pop: {np.sum(popArray)}")

        ###################### Coastal Adm1 ######################
        # Align using Warped VRT
        cFloodAligned_outputfile = vrtWarp(coastalFlood_path, cFlood_outputFolderPath,  
                                        vrt_options, countryCode, adm1Index, 'AlignedCoastalFlood')
        # At this point the coastal flood file is aligned and has been cropped to the pop bounding box
        # It will be cropped to the adm1 boundaries after merging with the fluvail and pluvial files
        tPrint(f"{countryCode}_{adm1Index} - Coastal aligned")

        ###################### Flood mosaic adm1 ######################
        # Create flood mosaic
        combinedFloodList = [FluvialCropped_outputfile] + [PluvialCropped_outputfile] + [cFloodAligned_outputfile]            
        allFiles = []
        for fp in combinedFloodList:
            src = rs.open(fp)
            allFiles.append(src)

        # Merge flood files. Using max pixel by pixel method
        floodArray, out_trans = merge(allFiles, method='max')
        out_meta = allFiles[0].meta.copy()
        out_meta.update({"height": floodArray.shape[1], 
                          "width": floodArray.shape[2],
                          "transform": out_trans})
        # export
        floodNameMerged = f"FloodMerged_{countryCode}_{adm1Index}.tif"
        FloodMerged_outputfile = export_raster_to_geotiff(floodArray, out_meta, Flood_outputFolderPath, floodNameMerged, 'FloodMerged')
        tPrint(f'{countryCode}_{adm1Index} - Flood mosaic merged & exported') 

        # ###################### Crop merged file flood ######################
        # crop
        floodArray, out_meta = crop_to_shp(FloodMerged_outputfile, adm1, 'FloodCropped')
        # export
        floodName = f"Flood_{countryCode}_{adm1Index}.tif"
        FloodCropped_outputfile = export_raster_to_geotiff(floodArray, out_meta, Flood_outputFolderPath, 
                                                            floodName, 'FloodCropped')
        tPrint(f'{countryCode}_{adm1Index} - Confirming same shape. popArray shape: {popArray.shape} floodArray shape: {floodArray.shape}')

        ###################### Categorize by flood bins ######################
        floodArray = np.digitize(floodArray, flood_bins, right=True) # True to include right bin edge
        floodArray = floodArray.astype(dtype = np.int8, copy = False)
        tPrint(f"{countryCode}_{adm1Index} - Flood categorized") 

        ###################### Convert to bool int arrays ######################
        list_IntBool_floodCat = convert_to_bool_int_array(floodArray)
        del floodArray

        ###################### Multiply flood array by pop array ######################
        # Multiply each int bool categorized flood array by the population array
        flood_pop = multiply_array_list(list_IntBool_floodCat, popArray)
        del list_IntBool_floodCat

        ##################### Export flood pop raster ######################
        # update meta
        out_meta.update({"nodata": 0., 'count': numberCategories})
        # export
        FloodPopName = f"FloodPop_{countryCode}_{adm1Index}.tif"
        export_raster_to_geotiff(flood_pop, out_meta, Raster_outputFolderPath, FloodPopName, 'flood pop stack')

        # ###################### Calculate sum for each exposure array ######################
        Flood_pop_sums = array3d_sum(flood_pop)

        #################################### Add to df ####################################
        adm1 = sums_to_df(Flood_pop_sums, adm1)
        resultDF = resultDF.append(adm1)
        tPrint(f"{countryCode}_{adm1Index} - Sums added to df")
    else:
        tPrint(f"{countryCode}_{adm1Index} - no pop file")
    return resultDF

In [70]:
def adm1_loop(sub_df, result_df):
  for namedTuple in sub_df.itertuples():
    adm1 = sub_df.loc[[int(namedTuple[0])]]
    adm1 = adm1[['ADM0_NAME','geometry']] 
    global adm1Index # making adm1Index global 
    adm1Index = get_adm1_index(adm1)
    adm1_name = f'{countryCode}_{adm1Index}' #countryCode is global
    # skip bad_adm1
    global bad_adm1 
    if adm1_name in bad_adm1: #bad_adm1 is global
        tPrint(f'{countryCode}_{adm1Index} is bad')
    else:
        try: # main processing happens hhere
            result_df = adm1_processing(adm1, result_df) #result is global
        except rasterio.errors.WindowError:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 window error")
            continue
        except ValueError:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 value error")
            continue
        except:
            #bad_adm1.append(f'{countryCode}_{adm1Index}')
            tPrint(f"{countryCode}_{adm1Index} - adm1 other error")
            continue
  return result_df

In [71]:
# Function to calculate summary stats for arrays
def stats (array): 
    stats_dict={
   'min': array.min(),
   'mean': array.mean(),
   'median': np.median(array),
   'max': array.max()}
    return stats_dict

<a id='TOC'></a>
*Run all code until here. Click Run > Run All Above Selected Cell.  
Then select the relevant section below*

# Analysis Options:
- [Loop for 2019 files](#Loop_2019)
- [Single country for 2019 files](#Large_2019)
- [Loop for 2016 files](#Loop_2016)
- [Single country for 2016 files](#Large_2016)

<a id='Loop_2019'></a>
## Loop for 2019 files
Back to [Analysis Options](#TOC)

In [51]:
# 2019 bad countries - add ISO3 country code for any country that causes an issue
bad_countries_2019 = []

In [ ]:
for shp_name, flood_name in matching_dict_2019.items():  
    if flood_name == "not_found":
        None
        #print(f"{shp_name} does not have flood data")
    elif type(flood_name) == list and not flood_name == ['Fiji_east', 'Fiji_west']:
        print(f"{shp_name} has multiple datasets, need to investigate")
    else:
        # Skip processed countries
        processedCountries = result['ISO3'].unique()
        processedCountries = np.append(processedCountries, previouslyProcessed)

        sub = adm.loc[adm['ADM0_NAME'] == shp_name].copy()
        countryCode = sub['ISO3'].iloc[0]
        countryNameInfo = pycountry.countries.get(alpha_3=countryCode)
        
        if not countryCode in processedCountries and not countryCode in bad_countries_2019:
            tPrint(f'{countryCode}: {sub.shape[0]}') # start

            # Adm0 fluvial and pluvial processing
            Fluvial_outputfile, Pluvial_outputfile = adm0_processing_2019(flood_name)

            # Adm1 Loop:
            result = adm1_loop(sub, result)
                        
            tPrint(f"{countryCode} - Completed")  # end
        else:
            if countryCode in processedCountries:
                tPrint(f'{countryCode} already processed')

<a id='Large_2019'></a>
## Single country for 2019 files 

Back to [Analysis Options](#TOC)

In [75]:
# Change country name here:
shp_name, flood_name = 'Vietnam', 'vietnam'

In [76]:
sub = adm.loc[adm['ADM0_NAME'] == shp_name].copy()

# Save country name
countryCode = sub['ISO3'].iloc[0]
countryNameInfo = pycountry.countries.get(alpha_3=countryCode)
tPrint(f'{countryCode}: {sub.shape[0]}') 

# Adm0 fluvial and pluvial processing
Fluvial_outputfile, Pluvial_outputfile = adm0_processing_2019(flood_name)

# # testing for one adm1 region
# sub = sub.iloc[[0]]
# Adm1 data processing loop:
result = adm1_loop(sub, result)

tPrint(f"{countryCode} - Completed")   

16:04:24	SSD: 10
16:04:24	SSD: No fluvial files found.
16:04:24	SSD: No pluvial files found.
16:04:24	SSD_1543 - adm1 other error
16:04:24	SSD_1544 - adm1 other error
16:04:24	SSD_1545 - adm1 other error
16:04:24	SSD_1546 - adm1 other error
16:04:24	SSD_1547 - adm1 other error
16:04:24	SSD_1548 - adm1 other error
16:04:24	SSD_1549 - adm1 other error
16:04:24	SSD_1550 - adm1 other error
16:04:24	SSD_1551 - adm1 other error
16:04:24	SSD_1552 - adm1 other error
16:04:24	SSD - Completed


<a id='Loop_2016'></a>
## Loop for 2016 files

Back to [Analysis Options](#TOC)

In [ ]:
# 2016 bad countries - add ISO3 country code for any country that causes an issue
bad_countries_2016 = []

In [ ]:
for shp_name, flood_name in matching_dict_2016.items():  
    if flood_name == "not_found":
        None
        #print(f"{shp_name} does not have flood data")
    elif type(flood_name) == list:
        print(f"{shp_name} has multiple datasets, need to investigate")
    else:
        # Skip processed countries
        processedCountries = result['ISO3'].unique()
        processedCountries = np.append(processedCountries, previouslyProcessed)

        sub = adm.loc[adm['ADM0_NAME'] == shp_name].copy()
        countryCode = sub['ISO3'].iloc[0]
        countryNameInfo = pycountry.countries.get(alpha_3=countryCode)
        
        if not countryCode in processedCountries and not countryCode in bad_countries_2016:
            tPrint(f'{countryCode}: {sub.shape[0]}')    

            # Adm0 fluvial and pluvial processing
            Fluvial_outputfile, Pluvial_outputfile = adm0_processing_2016(flood_name)  

            # Adm1 Loop:
            result = adm1_loop(sub, result)

            tPrint(f"{countryCode} - Completed")  # end
        else:
            if countryCode in processedCountries:
                tPrint(f'{countryCode} already processed')  

In [ ]:
result

In [ ]:
adm.loc[adm['ADM0_NAME'] == 'Cote ]

<a id='Large_2016'></a>
## Single country for 2016 files

Back to [Analysis Options](#TOC)

In [77]:
# Change country name here:
# shp_name, flood_name = 'South Korea', 'korea_south'
# shp_name, flood_name = 'Cote d\'Ivoire', 'cote_d_ivoire'
# shp_name, flood_name = 'Equatorial Guinea', 'equatorial_guinea'
# shp_name, flood_name = 'Iceland', 'iceland'
shp_name, flood_name = 'South Sudan', 'sudan'

In [78]:
sub = adm.loc[adm['ADM0_NAME'] == shp_name].copy()
countryCode = sub['ISO3'].iloc[0]
countryNameInfo = pycountry.countries.get(alpha_3=countryCode)
tPrint(f'{countryCode}: {sub.shape[0]}') 

# Adm0 fluvial and pluvial processing
Fluvial_outputfile, Pluvial_outputfile = adm0_processing_2016(flood_name)

# # testing for one adm1 region
# sub = sub.iloc[[0]]
# Loop:
result = adm1_loop(sub, result)

tPrint(f"{countryCode} - Completed")   

16:05:09	SSD: 10
16:05:09	SSD: No fluvial files found.
16:05:09	SSD: No pluvial files found.
16:05:09	SSD_1543 - adm1 other error
16:05:09	SSD_1544 - adm1 other error
16:05:09	SSD_1545 - adm1 other error
16:05:09	SSD_1546 - adm1 other error
16:05:09	SSD_1547 - adm1 other error
16:05:09	SSD_1548 - adm1 other error
16:05:09	SSD_1549 - adm1 other error
16:05:09	SSD_1550 - adm1 other error
16:05:09	SSD_1551 - adm1 other error
16:05:09	SSD_1552 - adm1 other error
16:05:09	SSD - Completed


In [ ]:
result

# Merging with adm & exporting 

In [115]:
# removing ISO3 col because it's in the results df
adm = adm[['ADM0_NAME', 'code', 'sample', 'region', 'geo_code2','geometry']]

In [116]:
# creating a merged df with only rows that have new results:
mergedDf1 = adm.merge(result, left_index=True, right_index=True)

In [117]:
# checking all looks good
mergedDf1.head()

,ADM0_NAME,code,sample,region,geo_code2,geometry,0-NoRiskPop,1-LowRiskPop,2-ModerateRiskPop,3-HighRiskPop,4-VeryHighRiskPop,5-WaterBodyPop,ISO3
1543,South Sudan,SSD,4 Lakes,SSA,SSD_2009_ADM1_2746,"POLYGON ((29.71243 7.82936, 29.72880 7.82471, ...",585609.62,83851.98,103949.76,78352.45,50030.95,999.75,SSD
1544,South Sudan,SSD,3 Unity,SSA,SSD_2009_ADM1_2747,"POLYGON ((30.79288 9.74854, 30.79076 9.74855, ...",187537.55,51406.49,75449.21,189957.36,450801.72,1470.76,SSD
1545,South Sudan,SSD,2 Central Equatoria,SSA,SSD_2009_ADM1_2748,"POLYGON ((31.69169 5.86563, 31.69272 5.86565, ...",1609954.38,75807.71,107257.20,52805.11,43922.11,1669.08,SSD
1546,South Sudan,SSD,3 Eastern Equatoria,SSA,SSD_2009_ADM1_2750,"POLYGON ((35.94770 4.62000, 35.94692 4.62000, ...",5114143.50,484394.88,462542.22,127622.18,18014.25,541.59,SSD
1547,South Sudan,SSD,2 Jonglei,SSA,SSD_2009_ADM1_2751,"POLYGON ((30.74804 9.46598, 30.75139 9.46796, ...",1068823.25,190713.08,256442.12,281408.31,236497.69,4892.21,SSD


In [118]:
# Exporting 
mergedDf1.to_file(Results_outputFolderPath / "SouthSudan.shp")